# Hero Video Processing - Google Colab

This notebook processes tennis videos with player tracking (SAM-3d-body) and ball detection (SAM3).

**Setup:**
1. Enable GPU: Runtime → Change runtime type → GPU (T4 or A100)
   - T4: Good for keypoints-only mode or lower resolution
   - A100: Recommended for full mesh rendering at higher resolutions
2. Make sure your folders are in `/content/drive/MyDrive/CourtVision/`:
   - SAM-3d-body/
   - SAM3/
   - models/
   - hero-video/ (with your raw video file)


## Step 1: Install Dependencies

In [ ]:
# Set CUDA memory allocation config to reduce fragmentation (MUST be before torch import)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
print("✅ Set PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True (helps prevent OOM errors)")

# Install core dependencies
!pip install -q opencv-python numpy torch torchvision torchaudio tqdm pillow
!pip install -q transformers accelerate

# Upgrade transformers to 5.0+ (required for SAM3)
!pip install --upgrade -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/facebookresearch/dinov2.git

# Install ALL SAM-3d-body dependencies (complete list from official INSTALL.md)
!pip install -q pytorch-lightning pyrender opencv-python yacs scikit-image einops timm dill pandas rich
!pip install -q hydra-core hydra-submitit-launcher hydra-colorlog pyrootutils webdataset chump
!pip install -q "networkx==3.2.1" roma joblib seaborn wandb appdirs appnope
!pip install -q ffmpeg cython jsonlines pytest xtcocotools loguru optree fvcore
!pip install -q black pycocotools tensorboard huggingface_hub
!pip install -q trimesh braceexpand

# Install Detectron2 (optional but recommended for human detection)
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git@a1ce2f9' --no-build-isolation --no-deps

# Install MoGe (required for FOV estimation)
!pip install -q git+https://github.com/microsoft/MoGe.git

# Install YOLO (ultralytics)
!pip install -q ultralytics

print("✅ All dependencies installed (including roma and all SAM-3d-body requirements)")

# Verify critical dependencies
print("\nVerifying critical dependencies...")
try:
    import roma
    print("✅ roma installed")
except ImportError:
    print("❌ roma NOT installed - this will cause errors!")

try:
    import torch
    print(f"✅ PyTorch {torch.__version__} installed")
    print(f"   CUDA available: {torch.cuda.is_available()}")
except ImportError:
    print("❌ PyTorch NOT installed")

try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__} installed")
except ImportError:
    print("❌ OpenCV NOT installed")

try:
    import transformers
    print(f"✅ Transformers {transformers.__version__} installed")
except ImportError:
    print("❌ Transformers NOT installed")

print("\n✅ Dependency verification complete")

## Step 2: Mount Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive

print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

print("\n✅ Google Drive mounted!")
print("   Your folders should be at: /content/drive/MyDrive/CourtVision/")

# Check what folders are available
import os
courtvision_path = '/content/drive/MyDrive/CourtVision'
if os.path.exists(courtvision_path):
    print(f"\n📁 Folders found in {courtvision_path}:")
    for item in os.listdir(courtvision_path):
        item_path = os.path.join(courtvision_path, item)
        if os.path.isdir(item_path):
            print(f"   ✅ {item}/")
        else:
            print(f"   📄 {item}")
else:
    print(f"\n⚠️ Path {courtvision_path} not found.")
    print("   Make sure your folders are in /CourtVision/ on your Drive")

## Step 3: Set Up Paths

In [ ]:
import sys
import os
from pathlib import Path
import cv2
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import json
from typing import List, Optional, Tuple, Dict, Any

# Set up Google Drive paths
drive_base = '/content/drive/MyDrive/CourtVision'

# Add SAM-3d-body to path
sam3d_path = f"{drive_base}/SAM-3d-body"
if os.path.exists(f"{sam3d_path}/sam-3d-body"):
    sys.path.insert(0, f"{sam3d_path}/sam-3d-body")
sys.path.insert(0, sam3d_path)

# Add hero-video to path
hero_video_path = f"{drive_base}/hero-video"
sys.path.insert(0, hero_video_path)
sys.path.insert(0, drive_base)

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Create output directory
os.makedirs('/content/output', exist_ok=True)

# Check GPU memory
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    free = total - reserved
    gpu_name = torch.cuda.get_device_name(0)
    
    print(f"   GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved, {free:.2f} GB free, {total:.2f} GB total")
    
    if "A100" in gpu_name:
        print(f"   ✅ A100 detected! You have plenty of memory ({free:.2f} GB free).")
        print(f"   💡 Consider using higher quality settings:")
        print(f"      - Higher process_resolution (e.g., 1080 or 1280 instead of 720)")
        print(f"      - Enable ensemble ball detection for better accuracy")
        print(f"      - Process every frame (frame_skip=1) for smoother output")

# Create output directory
os.makedirs('/content/output', exist_ok=True)

print("\n✅ Paths configured")


## Step 4: Find Video File

The raw video should be in your `hero-video` folder.

In [ ]:
# Find video file in hero-video folder
# Ensure paths are defined
drive_base = '/content/drive/MyDrive/CourtVision'
hero_video_path = f"{drive_base}/hero-video"

video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.MP4', '.AVI', '.MOV', '.MKV']
input_video_path = None

if not os.path.exists(hero_video_path):
    print(f"❌ hero-video folder not found: {hero_video_path}")
else:
    print(f"📁 Searching for video in: {hero_video_path}")
    for file in os.listdir(hero_video_path):
        file_path = os.path.join(hero_video_path, file)
        # Skip directories and the processing script
        if os.path.isdir(file_path) or file == 'process_hero_video.py':
            continue
        if any(file.endswith(ext) for ext in video_extensions):
            input_video_path = file_path
            print(f"✅ Found video: {file}")
            print(f"   Path: {input_video_path}")
            break

    if not input_video_path:
        print("❌ No video file found in hero-video folder!")
        print(f"   Make sure your raw video file is in: {hero_video_path}")

## Step 4.5: Authenticate with Hugging Face (Required for SAM-3d-body)

**The SAM-3d-body model requires Hugging Face authentication.**

1. Go to https://huggingface.co/facebook/sam-3d-body-dinov3
2. Click "Agree and access repository" (request access if needed)
3. Go to https://huggingface.co/settings/tokens
4. Create a token (read access is enough)
5. Run the cell below and paste your token when prompted

In [ ]:
# Authenticate with Hugging Face
from huggingface_hub import login
import os

print("🔐 Hugging Face Authentication")

# Check if token is already set in environment
if 'HF_TOKEN' in os.environ:
    print("✅ Found HF_TOKEN in environment variables")
    print("✅ Authenticated with Hugging Face!")
else:
    print("\nPlease paste your Hugging Face token below.")
    print("Get your token from: https://huggingface.co/settings/tokens")
    print("\n💡 Tip: If you have a token file, you can also set it manually:")
    print("  import os")
    print("  os.environ['HF_TOKEN'] = 'your_token_here'")
    print("  from huggingface_hub import login")
    print("  login()")
    print("\n---")
    
    # Login (will prompt for token)
    try:
        login()
        print("\n✅ Authenticated with Hugging Face!")
    except Exception as e:
        print(f"\n❌ Authentication failed: {e}")

## Step 5: Load Models

This step loads:
- SAM-3d-body (player tracking)
- SAM3 (ball detection)
- YOLO human detector (for multi-person detection)
- Court detector (if model available)

In [ ]:
# Ensure paths are defined
import sys
import os
import torch
from pathlib import Path

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
    print(f"Using device: {device}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print(f"Using device: {device} (CPU mode - will be slow)")

drive_base = '/content/drive/MyDrive/CourtVision'
sam3d_path = f"{drive_base}/SAM-3d-body"
hero_video_path = f"{drive_base}/hero-video"

# Add SAM-3d-body paths to sys.path (if not already added)
sam3d_body_path = f"{sam3d_path}/sam-3d-body"
if sam3d_body_path not in sys.path:
    sys.path.insert(0, sam3d_body_path)
if sam3d_path not in sys.path:
    sys.path.insert(0, sam3d_path)
if hero_video_path not in sys.path:
    sys.path.insert(0, hero_video_path)

# Load SAM-3d-body
print("\n" + "="*60)
print("Loading SAM-3d-body model...")
print("="*60)

# Check if authenticated with Hugging Face
try:
    from huggingface_hub import whoami
    user_info = whoami()
    print(f"   ✅ Authenticated as: {user_info.get('name', 'user')}")
except Exception:
    print("   ⚠️ Not authenticated with Hugging Face")
    print("   Please run Step 4.5 to authenticate first")
    raise ImportError("Hugging Face authentication required")

# First, set up human detector for multi-person detection (BEFORE loading SAM-3d-body)
human_detector = None
yolo_model_path = f"{drive_base}/models/player/playersnball5.pt"
print(f"\n1. Setting up YOLO human detector for multi-person detection...")
print(f"   Looking for YOLO model at: {yolo_model_path}")
print(f"   File exists: {os.path.exists(yolo_model_path)}")

if os.path.exists(yolo_model_path):
    try:
        # Import YOLO human detector wrapper
        from yolo_human_detector import YOLOHumanDetector
        print(f"   Creating YOLOHumanDetector...")
        human_detector = YOLOHumanDetector(model_path=Path(yolo_model_path), device=device)
        print("   ✅ YOLO human detector loaded (for multi-person detection)")
    except Exception as e:
        print(f"   ⚠️ Could not load YOLO human detector: {e}")
        print("   Will process full image as single person (only 1 player will be detected)")
        import traceback
        traceback.print_exc()
        human_detector = None
else:
    print(f"   ⚠️ YOLO model not found at: {yolo_model_path}")
    print("   Will process full image as single person (only 1 player will be detected)")
    print("   To enable multi-person detection, place playersnball5.pt in models/player/")
    # Also check alternative locations
    alt_paths = [
        f"{drive_base}/old/models/player/playersnball5.pt",
    ]
    for alt_path in alt_paths:
        if os.path.exists(alt_path):
            print(f"   Found YOLO model at alternative location: {alt_path}")
            print(f"   Please copy it to: {yolo_model_path}")
            break
    human_detector = None

# Try to load FOV estimator (improves camera calibration and mesh alignment)
fov_estimator = None
print(f"\n2. Setting up FOV estimator (improves mesh alignment)...")
try:
    from tools.build_fov_estimator import FOVEstimator
    fov_estimator = FOVEstimator(name="moge2", device=device)
    print("   ✅ FOV estimator loaded")
except Exception as e:
    print(f"   ⚠️ FOV estimator not available: {e}")
    fov_estimator = None

# Load SAM-3d-body model
print(f"\n3. Loading SAM-3d-body model...")
try:
    # Import from SAM-3d-body package directly
    from sam_3d_body import load_sam_3d_body_hf
    from sam_3d_body.sam_3d_body_estimator import SAM3DBodyEstimator
    from sam_3d_body.visualization.skeleton_visualizer import SkeletonVisualizer
    from sam_3d_body.metadata.mhr70 import pose_info as mhr70_pose_info
    
    model, model_cfg = load_sam_3d_body_hf("facebook/sam-3d-body-dinov3", device=device)
    estimator = SAM3DBodyEstimator(
        sam_3d_body_model=model,
        model_cfg=model_cfg,
        human_detector=human_detector,  # Pass the human detector for multi-person detection!
        human_segmentor=None,
        fov_estimator=fov_estimator,  # Pass FOV estimator for better mesh alignment
    )
    skeleton_visualizer = SkeletonVisualizer(mhr70_pose_info)
    if human_detector:
        print("   ✅ SAM-3d-body loaded with YOLO human detector (multi-person detection enabled)")
    else:
        print("   ✅ SAM-3d-body loaded (without human detector - will process full image as single person)")
except Exception as e:
    print(f"   ❌ Error loading SAM-3d-body: {e}")
    import traceback
    traceback.print_exc()
    estimator = None

# Load SAM3 ball detector
print(f"\n4. Loading SAM3 ball detector...")
try:
    sam3_path = f"{drive_base}/SAM3"
    if sam3_path not in sys.path:
        sys.path.insert(0, sam3_path)
    
    # Check transformers version first
    import transformers
    transformers_version = transformers.__version__.split(".")[0]
    if int(transformers_version) < 5:
        print(f"   ⚠️ Transformers version {transformers.__version__} is too old (need 5.0+)")
        print("   Upgrading transformers (this may take a few minutes)...")
        import subprocess
        result = subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "git+https://github.com/huggingface/transformers"], 
                                capture_output=True, text=True)
        if result.returncode == 0:
            print("   ✅ Transformers upgraded! Please restart runtime (Runtime → Restart runtime) and run this cell again.")
            raise ImportError("Transformers upgraded - restart runtime")
        else:
            print(f"   ❌ Failed to upgrade transformers: {result.stderr}")
            raise ImportError("Could not upgrade transformers")
    
    from test_sam3_ball_detection import SAM3BallDetector
    ball_detector = SAM3BallDetector(model_path=sam3_path, device=device, use_transformers=True)
    print("   ✅ SAM3 ball detector loaded")
except Exception as e:
    print(f"   ❌ Error loading SAM3: {e}")
    print("   Note: SAM3 requires transformers 5.0+. If upgrade was attempted, restart runtime.")
    ball_detector = None

# Initialize court detector (try to load if model exists)
court_detector = None
print(f"\n5. Setting up court detector...")
try:
    # Try importing court detector - add multiple possible paths
    court_paths = [
        (f"{drive_base}/old/scripts/legacy/demos", 'court_demo'),
        (f"{drive_base}/old/src/core", 'tennis_CV'),
    ]
    CourtDetector = None
    for court_path, module_name in court_paths:
        print(f"   Checking: {court_path}")
        if not os.path.exists(court_path):
            print(f"     ✗ Path does not exist")
            continue
        
        # Check if the module file exists
        module_file = f"{court_path}/{module_name}.py"
        if not os.path.exists(module_file):
            print(f"     ✗ Module file not found: {module_file}")
            continue
        
        if court_path not in sys.path:
            sys.path.insert(0, court_path)
        try:
            if module_name == 'court_demo':
                from court_demo import CourtDetector
            else:
                from tennis_CV import CourtDetector
            print(f"   ✅ Imported CourtDetector from: {court_path}")
            break
        except ImportError as import_err:
            print(f"     ✗ Import failed: {import_err}")
            continue
    
    if CourtDetector is None:
        print(f"   ⚠️ Could not import CourtDetector from any location")
        print(f"   Court detection will be disabled")
        raise ImportError("CourtDetector not available")
    
    # Find court model
    court_model_path = None
    possible_paths = [
        f"{drive_base}/models/court/model_tennis_court_det.pt",
        f"{drive_base}/old/models/court/model_tennis_court_det.pt",
    ]
    
    print(f"   Searching for court model 'model_tennis_court_det.pt'...")
    for path in possible_paths:
        exists = os.path.exists(path)
        print(f"     {'✓' if exists else '✗'} {path}")
        if exists:
            court_model_path = path
            print(f"   ✅ Found court model at: {court_model_path}")
            break
    
    if court_model_path:
        court_config = {
            'input_width': 640,
            'input_height': 360,
            'low_threshold': 170,
            'min_radius': 10,
            'max_radius': 25,
            'use_refine_kps': True,
            'use_homography': True
        }
        court_detector = CourtDetector(model_path=court_model_path, config=court_config)
        if court_detector.model is not None:
            print("   ✅ Court detector ready")
        else:
            print("   ⚠️ Court detector model failed to load")
            court_detector = None
    else:
        print(f"   ⚠️ Court model not found. Searched in:")
        for path in possible_paths:
            print(f"     {'✓' if os.path.exists(path) else '✗'} {path}")
        court_detector = None
except Exception as e:
    print(f"   ⚠️ Court detector initialization failed: {e}")
    import traceback
    traceback.print_exc()
    court_detector = None

if court_detector is None:
    print("   Court detector not available (will skip court detection)")

print("\n" + "="*60)
print("✅ All models loaded!")
print("="*60)

## Step 6: Configure Processing

Adjust these parameters as needed:
- `frame_skip`: Process every Nth frame (1 = all frames, higher = faster)
- `process_resolution`: Downscale to this width for processing (0 = original, 720/1080 = faster)
- `keypoints_only`: False = full 3D mesh (slow but best quality), True = skeleton only (fast)
- `enable_court_detection`: Enable court line detection (requires court model)

In [ ]:
# Processing configuration
# With A100 GPU (40GB), you can use higher quality settings:
config = {
    'frame_skip': 1,  # Process every frame (1 = all frames, higher = faster)
    'fps': 30.0,  # Output video FPS
    'player_color': '#50C878',  # Emerald green
    'ball_color': '#50C878',  # Emerald green
    'trail_length': 30,  # Ball trajectory trail length
    'keypoints_only': False,  # False = full mesh, True = keypoints only (faster)
    'process_resolution': 1080,  # A100 can handle 1080p! (720 = faster, 1080 = better quality, 0 = original)
    'enable_court_detection': True,  # Enable court detection (requires court model in models/court/)
    'use_ensemble_ball': False,  # Set to True for better ball detection (uses more memory but A100 can handle it)
}

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## Step 6.5: Test Single Frame (Preview)

**Run this before processing the full video!**

Process and display a single frame to verify everything looks good:
- Players are detected correctly
- Mesh aligns with players
- Ball detection works
- Court detection works (if enabled)

If everything looks good, proceed to Step 7 to process the full video.

In [ ]:
import sys
import os
from pathlib import Path
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

# Ensure paths are defined
drive_base = '/content/drive/MyDrive/CourtVision'
hero_video_path = f"{drive_base}/hero-video"

# Add paths to sys.path
sys.path.insert(0, str(Path(hero_video_path).resolve()))
sys.path.insert(0, str(Path(f"{drive_base}/SAM-3d-body/sam-3d-body").resolve()))

# Check that required variables exist
if 'input_video_path' not in globals() or input_video_path is None:
    print("❌ Error: Video file not found. Please run Step 4 first.")
elif 'config' not in globals():
    print("❌ Error: Configuration not set. Please run Step 6 first.")
elif 'estimator' not in globals() or estimator is None:
    print("❌ Error: Models not initialized. Please run Step 5 first.")
else:
    print("✅ All prerequisites met. Processing single frame...\n")
    
    # Frame to test (adjust if needed - use a frame with visible player/ball)
    test_frame_number = 20  # Change this to test different frames
    
    # Open video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"❌ Error: Could not open video: {input_video_path}")
    else:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"📹 Video opened: {total_frames} total frames")
        print(f"🎯 Testing frame {test_frame_number} (0-indexed: {test_frame_number-1})\n")
        
        # Seek to frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, test_frame_number - 1)
        ret, frame = cap.read()
        
        if not ret:
            print(f"❌ Error: Could not read frame {test_frame_number}")
        else:
            print(f"✅ Frame {test_frame_number} loaded: {frame.shape}")
            
            # Apply same processing as main script
            process_resolution = config.get('process_resolution', 720)
            original_height, original_width = frame.shape[:2]
            
            # Calculate scale factor
            if process_resolution > 0:
                scale_factor = process_resolution / original_width
                process_width = process_resolution
                process_height = int(original_height * scale_factor)
            else:
                scale_factor = 1.0
                process_width = original_width
                process_height = original_height
            
            # Downscale if needed
            if scale_factor < 1.0:
                print(f"📐 Downscaling from {original_width}x{original_height} to {process_width}x{process_height}")
                frame = cv2.resize(frame, (process_width, process_height), interpolation=cv2.INTER_AREA)
            
            # Convert BGR to RGB for SAM-3d-body
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Process with SAM-3d-body
            print("🤖 Running SAM-3d-body...")
            keypoints_only = config.get('keypoints_only', False)
            # Check if detector is being used
            if hasattr(estimator, 'detector') and estimator.detector is not None:
                print(f"   ✅ Using human detector: {type(estimator.detector).__name__}")
            else:
                print(f"   ⚠️ No human detector - will process full image as single person")
            # Use lower thresholds for better multi-person detection
            outputs = estimator.process_one_image(
                frame_rgb,
                inference_type="full" if not keypoints_only else "keypoints_only",
                bbox_thr=0.15,  # Even lower threshold to detect more people
                nms_thr=0.5,   # Higher NMS to keep separate people
            )
            print(f"✅ SAM-3d-body: {len(outputs)} person(s) detected")
            
            # Process with ball detector
            print("⚽ Running ball detection...")
            ball_prompt = "tennis ball"
            ball_detection = None
            try:
                if hasattr(ball_detector, 'detect_ball'):
                    import inspect
                    sig = inspect.signature(ball_detector.detect_ball)
                    if 'threshold' in sig.parameters:
                        ball_detection = ball_detector.detect_ball(
                            frame,
                            text_prompt=ball_prompt,
                            threshold=0.3
                        )
                    else:
                        ball_detection = ball_detector.detect_ball(frame, text_prompt=ball_prompt)
                    
                    if ball_detection:
                        center, conf, mask = ball_detection
                        print(f"✅ Ball detected at {center} (confidence: {conf:.2f})")
                    else:
                        print("⚠️ No ball detected")
            except Exception as e:
                print(f"⚠️ Ball detection error: {e}")
            
            # Create visualization
            print("🎨 Creating visualization...")
            from visualizer import HeroVideoVisualizer
            
            # Convert hex colors to BGR tuples (OpenCV format)
            def hex_to_bgr(hex_color: str):
                """Convert hex color to BGR tuple for OpenCV."""
                hex_color = hex_color.lstrip('#')
                r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
                return (b, g, r)  # BGR format
            
            player_color_bgr = hex_to_bgr(config['player_color'])
            ball_color_bgr = hex_to_bgr(config['ball_color'])
            
            visualizer = HeroVideoVisualizer(
                player_color=player_color_bgr,
                ball_color=ball_color_bgr,
                trail_length=config['trail_length']
            )
            
            # Simple ball trajectory for preview (just current detection)
            ball_trajectory = []
            if ball_detection:
                center, _, _ = ball_detection
                ball_trajectory = [center]  # Just show current position
            
            # Try court detection if available
            court_keypoints = None
            if config.get('enable_court_detection', False):
                try:
                    if 'court_detector' in globals() and court_detector is not None:
                        print("🏟️ Running court detection...")
                        court_keypoints = court_detector.detect_court_in_frame(frame)
                        if court_keypoints:
                            # Filter out None points
                            court_keypoints = [
                                (kp[0], kp[1]) if kp and kp[0] is not None and kp[1] is not None else None
                                for kp in court_keypoints
                            ]
                            valid_points = sum(1 for kp in court_keypoints if kp is not None)
                            print(f"✅ Court detected: {valid_points} valid keypoints")
                        else:
                            print("⚠️ Court detection returned None")
                    else:
                        print("⚠️ Court detector not available (not initialized in Step 5)")
                except Exception as e:
                    print(f"⚠️ Court detection error: {e}")
            
            # Create frame with skeleton
            vis_frame = visualizer.create_frame(
                frame=frame,
                player_outputs=outputs,
                ball_detection=ball_detection,
                ball_trajectory=ball_trajectory,
                skeleton_visualizer=None,
                court_keypoints=court_keypoints
            )
            
            # Add mesh if available and not keypoints_only
            total_start = time.time()
            
            if not keypoints_only and len(outputs) > 0:
                try:
                    from mesh_visualizer import visualize_sample_together_emerald
                    if hasattr(estimator, 'faces') and estimator.faces is not None:
                        print("🎨 Adding 3D mesh...")
                        print(f"   ⚠️ WARNING: Full mesh rendering takes 2-4 minutes per frame!")
                        print(f"   This is normal - mesh rendering is very computationally expensive")
                        mesh_start_time = time.time()
                        vis_frame_rgb = cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB)
                        faces = estimator.faces
                        print(f"   Starting mesh visualization (faces shape: {faces.shape})...")
                        mesh_frame_rgb = visualize_sample_together_emerald(vis_frame_rgb, outputs, faces)
                        vis_frame = cv2.cvtColor(mesh_frame_rgb.astype(np.uint8), cv2.COLOR_RGB2BGR)
                        mesh_time = time.time() - mesh_start_time
                        print(f"✅ Mesh added (took {mesh_time:.1f} seconds = {mesh_time/60:.1f} minutes)")
                    else:
                        print("⚠️ No faces available in estimator - mesh rendering skipped")
                        print("   This means only skeleton will be shown (keypoints-only mode)")
                except Exception as e:
                    print(f"⚠️ Mesh rendering failed: {e}")
                    import traceback
                    traceback.print_exc()
            elif keypoints_only:
                print("ℹ️ Keypoints-only mode - mesh rendering skipped (faster, ~0.5s/frame)")
            elif len(outputs) == 0:
                print("ℹ️ No player detections - mesh rendering skipped")
            
            total_time = time.time() - total_start
            print(f"\n⏱️ Total processing time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
            
            # Upscale back to original if we downscaled
            if scale_factor < 1.0:
                vis_frame = cv2.resize(vis_frame, (original_width, original_height), interpolation=cv2.INTER_LINEAR)
            
            # Display result
            print("\n✅ Processing complete! Displaying result...\n")
            
            # Convert BGR to RGB for display
            vis_frame_rgb = cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB)
            
            # Create figure
            fig, axes = plt.subplots(1, 2, figsize=(20, 10))
            
            # Original frame
            axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f'Original Frame {test_frame_number}', fontsize=14)
            axes[0].axis('off')
            
            # Processed frame
            axes[1].imshow(vis_frame_rgb)
            axes[1].set_title(f'Processed Frame {test_frame_number} (with mesh, ball, skeleton)', fontsize=14)
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print(f"\n📊 Frame Info:")
            print(f"   Original size: {original_width}x{original_height}")
            print(f"   Processed size: {vis_frame.shape[1]}x{vis_frame.shape[0]}")
            print(f"   Players detected: {len(outputs)}")
            print(f"   Ball detected: {'Yes' if ball_detection else 'No'}")
            print(f"   Court detected: {'Yes' if court_keypoints else 'No'}")
            print(f"   Mesh rendering: {'Yes' if not keypoints_only else 'No (keypoints only)'}")
            print(f"\n✅ If this looks good, proceed to Step 7 to process the full video!")
        
        cap.release()

## Step 7: Process Full Video

**⚠️ This will take a long time!**

Processing times:
- Keypoints-only: ~0.5 seconds per frame
- Full mesh at 1080px: ~3-4 minutes per frame
- Full mesh at 720px: ~2-3 minutes per frame

For a 30fps video (6518 frames):
- Keypoints-only: ~54 minutes
- Full mesh at 1080px: ~325-434 hours (13-18 days!)
- Full mesh at 720px: ~217-326 hours (9-13 days!)

**Recommendation:** Use `frame_skip=5` or higher for full mesh to reduce processing time significantly.

In [ ]:
import subprocess
import sys
import os

# Ensure paths are defined
drive_base = '/content/drive/MyDrive/CourtVision'
hero_video_path = f"{drive_base}/hero-video"

# Check that required variables exist
if 'input_video_path' not in globals() or input_video_path is None:
    print("❌ Error: Video file not found. Please run Step 4 first.")
elif 'config' not in globals():
    print("❌ Error: Configuration not set. Please run Step 6 first.")
else:
    # Find processing script
    hero_video_script = f"{hero_video_path}/process_hero_video.py"

    if not os.path.exists(hero_video_script):
        print(f"❌ Script not found: {hero_video_script}")
    else:
        print(f"✅ Found script: {hero_video_script}")
        print("\n🚀 Processing video...")
        
        script_dir = os.path.dirname(hero_video_script)
        
        cmd = [
            sys.executable, hero_video_script,
            "--input", input_video_path,
            "--output", "/content/output/hero_video_processed.mp4",
            "--frame-skip", str(config['frame_skip']),
            "--fps", str(config['fps']),
            "--player-color", config['player_color'],
            "--ball-color", config['ball_color'],
            "--trail-length", str(config['trail_length']),
        ]
        
        if config['keypoints_only']:
            cmd.append("--keypoints-only")
        
        if config['process_resolution'] > 0:
            cmd.extend(["--process-resolution", str(config['process_resolution'])])
        
        if config['enable_court_detection']:
            cmd.append("--enable-court-detection")
        
        if config['use_ensemble_ball']:
            cmd.append("--ensemble-ball")
        
        print("Command:")
        print(" ".join(cmd))
        print("\n⏳ Processing (this will take a while)...")
        print("📊 Debug output will stream below in real-time:\n")
        
        import time
        start_time = time.time()
        
        # Run subprocess with real-time output streaming
        process = subprocess.Popen(
            cmd,
            cwd=script_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # Combine stderr into stdout
            text=True,
            bufsize=1,  # Line buffered
            universal_newlines=True
        )
        
        # Stream output in real-time
        output_lines = []
        for line in process.stdout:
            print(line, end='')  # Print immediately
            output_lines.append(line)
            sys.stdout.flush()  # Force flush to show output immediately
        
        # Wait for process to complete
        returncode = process.wait()
        end_time = time.time()
        
        # Get full output
        full_output = ''.join(output_lines)
        
        if returncode == 0:
            print(f"\n\n✅ Processing complete!")
            print(f"⏱️ Time: {(end_time - start_time) / 60:.2f} minutes")
            output_video_path = "/content/output/hero_video_processed.mp4"
        else:
            print(f"\n\n❌ Processing failed with return code {returncode}")
            print("\n📋 Full output (last 200 lines):")
            output_lines_list = full_output.split('\n')
            for line in output_lines_list[-200:]:
                print(line)
            output_video_path = None

## Step 8: Download Output Video

In [ ]:
from google.colab import files

output_path = "/content/output/hero_video_processed.mp4"
if os.path.exists(output_path):
    print(f"📥 Downloading {output_path}...")
    files.download(output_path)
    print("✅ Download complete!")
else:
    print(f"❌ Output file not found: {output_path}")